In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn import tree
import numpy as np
import pandas as pd
from sklearn import metrics

In [2]:
data = pd.read_csv('/home/dominik/Dokumenty/Studia/Data-mining/Lista5-trees/bank-additional/bank-additional/bank-additional-full.csv', sep=';')
data.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [3]:
np.any(data.isnull())

False

In [4]:
data = data.drop('duration', axis=1)

In [5]:
data.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [6]:
numData = data.copy()

for name in data.columns.values:
    if data[name].dtype == 'object':
        numData[name] = pd.Categorical(data[name]).codes

In [7]:
numData.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,3,1,0,0,0,0,1,6,1,1,999,0,1,1.1,93.994,-36.4,4.857,5191.0,0
1,57,7,1,3,1,0,0,1,6,1,1,999,0,1,1.1,93.994,-36.4,4.857,5191.0,0
2,37,7,1,3,0,2,0,1,6,1,1,999,0,1,1.1,93.994,-36.4,4.857,5191.0,0
3,40,0,1,1,0,0,0,1,6,1,1,999,0,1,1.1,93.994,-36.4,4.857,5191.0,0
4,56,7,1,3,0,0,2,1,6,1,1,999,0,1,1.1,93.994,-36.4,4.857,5191.0,0


In [8]:
data[[i for i in data.columns.values if data[i].dtype != 'object']].describe()

,age,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
count,41188.00000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000
mean,40.02406,2.567593,962.475454,0.172963,0.081886,93.575664,-40.502600,3.621291,5167.035911
std,10.42125,2.770014,186.910907,0.494901,1.570960,0.578840,4.628198,1.734447,72.251528
min,17.00000,1.000000,0.000000,0.000000,-3.400000,92.201000,-50.800000,0.634000,4963.600000
25%,32.00000,1.000000,999.000000,0.000000,-1.800000,93.075000,-42.700000,1.344000,5099.100000
50%,38.00000,2.000000,999.000000,0.000000,1.100000,93.749000,-41.800000,4.857000,5191.000000
75%,47.00000,3.000000,999.000000,0.000000,1.400000,93.994000,-36.400000,4.961000,5228.100000
max,98.00000,56.000000,999.000000,7.000000,1.400000,94.767000,-26.900000,5.045000,5228.100000


In [32]:
mask = np.random.uniform(0,1, len(numData))
train_data = numData[mask <= 0.7]
test_data = numData[mask > 0.7]

y = train_data['y']
X = train_data.drop('y', axis=1)
print(X.shape)

t = tree.DecisionTreeClassifier(max_leaf_nodes=30, max_depth=15, min_samples_split=10)
t = t.fit(X,y)

y = test_data['y']
X = test_data.drop('y', axis=1)
print(X.shape)


t.score(X,y)

(28934, 19)
(12254, 19)


0.90166476252652195

In [33]:
importances = t.tree_.compute_feature_importances()
importances

array([ 0.01662366,  0.        ,  0.00334078,  0.        ,  0.        ,
        0.0033787 ,  0.        ,  0.01926784,  0.04035975,  0.0293644 ,
        0.        ,  0.02012216,  0.00864743,  0.11182917,  0.01014579,
        0.        ,  0.06151122,  0.0466132 ,  0.62879589])

In [43]:
mask = np.random.uniform(0,1, len(numData))
train_data = clearedData[mask <= 0.5]
test_data = clearedData[mask > 0.5]
np.sum(train_data['y'] == 1)
trainnp = np.array(train_data)
train = np.concatenate((trainnp[trainnp[:,-1] == 1,:],trainnp[trainnp[:,-1] != 1,:][:15310,:]))

In [53]:
y = train[:,-1]
X = train[:,:-1]
# print(X.shape)

f = RandomForestClassifier(n_estimators=50, max_leaf_nodes=20, max_depth=10, min_samples_split=10,class_weight={0:1, 1:3})
f = f.fit(X,y)

y = test_data['y']
X = test_data.drop('y', axis=1)
# print(X.shape)

print(f.score(X,y))

confusion_matrix = metrics.confusion_matrix(y, f.predict(X))
print ("Confusion Matrix\n", confusion_matrix)

0.810554803789
Confusion Matrix
 [[15286  3068]
 [  852  1486]]


In [28]:
y = train_data['y']
X = train_data.drop('y', axis=1)
# print(X.shape)

f = RandomForestClassifier(n_estimators=50, max_leaf_nodes=20, max_depth=10, min_samples_split=10)
f = f.fit(X,y)

y = test_data['y']
X = test_data.drop('y', axis=1)
# print(X.shape)


print(f.score(X,y))

confusion_matrix = metrics.confusion_matrix(y, f.predict(X))
print ("Confusion Matrix\n", confusion_matrix)

0.900541533701
Confusion Matrix
 [[18175   190]
 [ 1867   450]]


In [34]:
clearMask = [i for i in range(len(importances)) if importances[i] < 0.01]
clearedData = numData.drop(numData.columns.values[clearMask],axis=1)
clearedData.head()

,age,contact,month,day_of_week,pdays,poutcome,emp.var.rate,cons.conf.idx,euribor3m,nr.employed,y
0,56,1,6,1,999,1,1.1,-36.4,4.857,5191.0,0
1,57,1,6,1,999,1,1.1,-36.4,4.857,5191.0,0
2,37,1,6,1,999,1,1.1,-36.4,4.857,5191.0,0
3,40,1,6,1,999,1,1.1,-36.4,4.857,5191.0,0
4,56,1,6,1,999,1,1.1,-36.4,4.857,5191.0,0


In [49]:
mask = np.random.uniform(0,1, len(clearedData))
train_data = clearedData[mask <= 0.7]
test_data = clearedData[mask > 0.7]

y = train_data['y']
X = train_data.drop('y', axis=1)
print(X.shape)

t = tree.DecisionTreeClassifier(max_leaf_nodes=35, max_depth=15, min_samples_split=8)
t = t.fit(X,y)

y = test_data['y']
X = test_data.drop('y', axis=1)
print(X.shape)


t.score(X,y)

(28856, 9)
(12332, 9)


0.90293545248134932

In [50]:
confusion_matrix = metrics.confusion_matrix(y, t.predict(X))
print ("Confusion Matrix\n", confusion_matrix)

Confusion Matrix
 [[10762   195]
 [ 1002   373]]


In [51]:
from sklearn.metrics import classification_report
print(classification_report(y, 
                            t.predict(X), 
                            target_names=['0', '1']))

             precision    recall  f1-score   support

          0       0.91      0.98      0.95     10957
          1       0.66      0.27      0.38      1375

avg / total       0.89      0.90      0.88     12332



In [68]:
mask = np.random.uniform(0,1, len(clearedData))
train_data = clearedData[mask <= 0.7]
test_data = clearedData[mask > 0.7]

y = train_data['y']
X = train_data.drop('y', axis=1)
print(X.shape)

f = RandomForestClassifier(n_estimators=50, max_leaf_nodes=20, max_depth=10, min_samples_split=10,class_weight={0:1, 1:2})
f = f.fit(X,y)

y = test_data['y']
X = test_data.drop('y', axis=1)
print(X.shape)


print(f.score(X,y))
confusion_matrix = metrics.confusion_matrix(y, f.predict(X))
print ("Confusion Matrix\n", confusion_matrix)

(28755, 10)
(12433, 10)
0.885466098287
Confusion Matrix
 [[10333   684]
 [  740   676]]


In [58]:
print(classification_report(y, 
                            f.predict(X), 
                            target_names=['0', '1']))

             precision    recall  f1-score   support

          0       0.91      0.99      0.95     11096
          1       0.67      0.21      0.32      1358

avg / total       0.88      0.90      0.88     12454



In [47]:
clearedData['train'] = np.random.uniform(0, 1, len(clearedData))
err = 0
for i in range(10):
    data_test = clearedData[(clearedData['train'] >= i*0.1) & (clearedData['train'] < (i+1)*0.1)]
    data_train = clearedData[(clearedData['train'] < i*0.1) | (clearedData['train'] >= (i+1)*0.1)]

    y = data_train['y']
    X = data_train.drop('y', axis = 1).drop('train',axis=1)

    t = tree.DecisionTreeClassifier(max_leaf_nodes=30, max_depth=15, min_samples_split=10)
    t = t.fit(X, y)
#     print('depth', [i.tree_.max_depth for i in t.estimators_])
    y = data_test['y']
    X = data_test.drop('y', axis = 1).drop('train',axis=1)
    print(t.score(X,y))
    err += t.score(X,y) / 10

print('Average score: ',err)

0.904622157007
0.896509936985
0.899120664387
0.901291737753
0.900320750062
0.907737368806
0.894109229668
0.899106927347
0.9071863581
0.899402628435
Average score:  0.900940775855


In [48]:
clearedData['train'] = np.random.uniform(0, 1, len(clearedData))
err = 0
for i in range(10):
    data_test = clearedData[(clearedData['train'] >= i*0.1) & (clearedData['train'] < (i+1)*0.1)]
    data_train = clearedData[(clearedData['train'] < i*0.1) | (clearedData['train'] >= (i+1)*0.1)]

    y = data_train['y']
    X = data_train.drop('y', axis = 1).drop('train',axis=1)

    f = RandomForestClassifier(n_estimators=10, max_leaf_nodes=20, max_depth=10, min_samples_split=10)
    f = f.fit(X, y)
#     print('depth', [i.tree_.max_depth for i in t.estimators_])
    y = data_test['y']
    X = data_test.drop('y', axis = 1).drop('train',axis=1)
    print(f.score(X,y))
    err += f.score(X,y) / 10

print('Average score: ',err)

0.903700073511
0.893317132442
0.902463299328
0.901447921532
0.903485254692
0.898139079334
0.904702970297
0.891819941917
0.900310040544
0.898526214061
Average score:  0.899791192766
